In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import mwparserfromhell
from dgnutils import *
# sys.path.append("/Users/nish/development/git/etymology_explorer/modules")
sys.path.append("/home/ubuntu/projects/etymology_explorer/modules")
from ety_utils import * # imports wikiextractor
u = os.environ['RDS_ETY_USER']; p = os.environ['RDS_ETY_PASSWORD']; h = os.environ['RDS_ETY_HOST']
session=requests.session()
train_conn, train_cursor = connect('training_data', user=u, password=p, host=h)
dev_conn, dev_cursor = connect('etymology_explorer_dev', user=u, password=p, host=h)
# test_conn, test_cursor = connect('etymology_explorer_test', user=u, password=p, host=h)
# stage_conn, stage_cursor = connect('etymology_explorer_staging', user=u, password=p, host=h)

1/6/21 dgnutils update loaded! Added more mysql db functions


In [ ]:
# Get these to pass!
# preprocess_etymology('From banana and thus cognates with banan.') == 'From banana.'
# preprocess_etymology('From {{test}}, from {{test}}, and related to {{test}}; test') == 'From {{test}}, from {{test}}; test'

# Full load test

In [ ]:
# Create test xml
# head -176497 enwiktionary-latest-pages-articles.xml > test.xml
# echo "</mediawiki>" >> test.xml
# wc -l test.xml
# tail -100 test.xml
# cd ..
# ./WikiExtractor.py --processes 4 -o output_test --json --no_templates --lists --sections input/test.xml

In [ ]:
wp = WikiProcessor(
    '/home/ubuntu/projects/wikiextractor/', 
    channel='test', # Test makes self.test = true
#     channel='dev', 
    store_intermediates=True,
)

In [ ]:
wp.load_wl_2_id_values()

In [ ]:
processed_wikidump = wp.load_wikidump_etymologies(); len(processed_wikidump)

In [ ]:
en_etys_dl, wp.en_dict = wp.create_and_insert_mysql_entries(processed_wikidump) # no cache

In [ ]:
test_cursor.e('SELECT COUNT(*) FROM etymologies')

# Add links to etymologies 

In [ ]:
# Do a hack
# Go through all new etymologies, and find instances where the expanded text matches to one of the existing etymologies, then add that to the etymology
# Search through each unmatched etymology, look for (language, word) that links to an _id.

In [ ]:
# Manually get all the etymologies
# DO NOT RESET DATABASE
wp = WikiProcessor('/home/ubuntu/projects/wikiextractor/', channel='staging', )
wp.load_wl_2_id_values()
wp.load_language_dict()
log_level('d')

## Get en_etys_dl

In [ ]:
wp.load_wl_2_id_values() # 17s
wp.load_language_dict() #1s

In [ ]:
processed_wikidump = wp.load_wikidump_etymologies(); len(processed_wikidump) # 1.5m on r4.2xl

In [ ]:
en_conns_dl, en_etys_dl, en_pos_dl, en_prons_dl, en_defs_dl, etys_dl = wp.create_mysql_data_from_processed_wikiextraction_data(processed_wikidump, log=True)

In [ ]:
parsed_etymologies_except_conns = multi_parse_wikitext_sentences(
    [e['wikitext'] for e in en_etys_dl], 
    cache_file=wp.cache_path+'ety.wik' if wp.cache_path else None,
    ignore_connection_forming=True,
)
en_etys_dl = [{**z[0], 'wikitext':z[1]} for z in zip(en_etys_dl, parsed_etymologies_except_conns)]

In [ ]:
parsed_etymologies = multi_parse_wikitext_sentences(
    [e['wikitext'] for e in en_etys_dl], 
    cache_file=wp.cache_path+'ety.wik' if wp.cache_path else None,
    ignore_connection_forming=False,
)
en_etys_dl = [{**z[0], 'etymology':z[1]} for z in zip(en_etys_dl, parsed_etymologies)]

## start from en_etys_dl

In [ ]:
# Checkpoint
# pickle.dump(en_etys_dl, open('en_etys_dl', 'wb+'))
en_etys_dl = pickle.load(open('en_etys_dl', 'rb'))

In [ ]:
len(en_etys_dl), en_etys_dl[0]

In [ ]:
wp.wl_2_id[('dictionarium', 'Latin')]

In [ ]:
wp.getNodesFromTemplate('{{bor|en|ML.|dictionarium}}', 'root', allow_non_connections=True)
# getTemplateInfo('{{bor|en|ML.|dictionarium}}')

In [ ]:
templates = mwparserfromhell.parse(en_etys_dl[0]['wikitext']).filter_templates(); templates
[wp.getNodesFromTemplate(t, 'root') for t in templates]

In [ ]:
for en_ety in en_etys_dl:
    
wp.load

In [ ]:
# Check for multiple entries

In [ ]:
[a['etymology'] for a in all_entry_etymologies if 'Borrowed from Medieval Latin dictionarium' in a['etymology']]

In [ ]:
processed_wikidump

In [ ]:
# 12.8s
all_entry_etymologies = stage_cursor.d('SELECT entry_id, etymology FROM entry_etymologies')

# Get entries info from wikitext

In [ ]:
# Get all the wikitext entries (for now just a sample)

# Find ones that have a gloss, or definition
# Find the word and language for that wikitext, then the id
# See if that id has an existing definition or entry

# Improve interwikitext parsing

- Reduce the interwiki text to the important information. i.e. remove unhelpful information
- Grab all interwikitext
- Make a plot of the frequencies for reference
- Reduce them to the most common with regex etc
- Count
  - Staring: 121506
  - after stranded punctuation fix: 119934
  - after another comma fix: 119886
  - after removing cognates: 114426 

In [ ]:
sql_stmt = """ 
SELECT _id, word, language_name, ee.entry_id, wikitext FROM entry_etymologies ee
INNER JOIN entry_connections ec ON ee.entry_id = ec.entry_id
INNER JOIN etymologies e ON e._id = ec.etymology_id """
    # sql_stmt += ' LIMIT 5' # for testing
    # sql_stmt += ' WHERE _id = 204484 '

In [ ]:
# wikitext_data = dev_cursor.d(sql_stmt) # 90s - 120s
# pickle.dump(wikitext_data, open('wikitext_data', 'wb+'))
wikitext_data = pickle.load(open('wikitext_data', 'rb')) # 2s

In [ ]:
sentiment_data={} # 80s 120s
for wi in wikitext_data:
    for g in get_wikitext_parts_dict(**wi).values():
        if g['preceding_text'] == '' and g['place'] == 0: continue
        sentiment_data.setdefault(g['preceding_text'],{}).setdefault('entry_ids', []).append(wi['entry_id'])
        if not 'sentiment_type' in sentiment_data[g['preceding_text']]:
            sentiment_data[g['preceding_text']]['sentiment_type'] = get_sentiment_type(g['preceding_text']) 
len(sentiment_data)

In [ ]:
etymology = 'As a verb, {{test}} from [[test]] {{inh{{test}}|en|enm|springen||to burst or flow forth, to sprout, to emerge, to happen, to become known, to sprinkle}}, from {{inh|en|ang|springan||to burst or flow forth, to sprout, to emerge, to become known}}, cognate with {{cog|af|spring}}, {{cog|fy|springe}}, Dutch & German {{m|nl|springen}}, {{cog|da|springe}}, {{cog|sv|springa}}. Further etymology is uncertain, but usually taken to derive from a __wiki__ verb reconstructed as {{m|gem-pro|*springaną||to burst forth}}, from a __wiki__ root reconstructed {{m|ine-pro|*sperǵʰ-}} whose other descendants may include {{cog|lt|spreñgti||to push (in)}}, {{cog|cu|прѧсти||to spin, to stretch}}, {{cog|la|spargere||to sprinkle, to scatter}}, {{cog|grc|σπέρχω||to hasten}}, {{cog|sa|स्पृहयति|tr=spṛháyati||to be eager}}. Some newer senses derived from the noun. As a noun, from {{inh|en|enm|spring||a wellspring, tide, branch, sunrise, kind of dance or blow, ulcer, snare, flock}}, from {{inh|en|ang|spring||wellspring, ulcer}} and {{inh|en|ang|spryng||a jump}}, from ablaut forms of the __wiki__ verb. Further senses derived from the verb and from clippings of {{m|en|day-spring}}, {{m|en|springtime}}, {{m|en|spring tide}}, etc. Its sense as the season, first attested in a work predating 1325, gradually replaced Old English {{m|ang|lencten||spring, Lent}} as that word became more specifically liturgical. Compare {{m|en|fall}}.'
m = mwparserfromhell.parse(etymology)
wiki_dict = {str(m)[2:-2]:i for i,m in enumerate(list(m.filter_templates(recursive=False)))}
wiki_dict
for wiki,index in sorted(wiki_dict.items(), key=lambda x: len(x[0]), reverse=True): # sort by length
    etymology = etymology.replace(wiki, str(index))
etymology

In [ ]:
def protect_wikis(text):
    """
    replace wikitext {{test|text}} with {{0}} and store the text in a dict
    """
    wiki_dict = {}
    for 
    
etymology, protect_wikis_dict = def (etymology)

In [ ]:
# assert fix_broken_punctuation('{{cog|crh|kim}} ,{{cog|krc|ким|tr=kim}}') == '{{cog|crh|kim}}, {{cog|krc|ким|tr=kim}}'
# preprocess_etymology('From {{inh|kdr|trk-pro|*kem|gloss=who}}. Relate to {{cog|crh|kim}} ,{{cog|krc|ким|tr=kim}} ,{{cog|kum|ким|tr=kim}} , {{cog|uum|ким|tr=kim}} ,{{cog|kim|ӄум|tr=qum}}, {{cog|tyv|кым}}, etc.')

In [ ]:
# remove everything after ". Cognate"
# handle "" words
# remove duplicated punctuation like ", , "
# fix "(spore):, " Danish spore

In [ ]:
get_entry_info(dev_conn, enid)

In [ ]:
enid = 2012
w = next(iter(get_entry_info(dev_conn, enid)['words'].values()))['entries'][enid]['wikitext']
print(w), print(), print(preprocess_etymology(w));

In [ ]:
a = r'([Rr]elate|[Cc]ognate)[^.;!?]*([^.;!?]|$)'
re.sub(a, '', 'From {{test}} and related to banana. From {{something}} else')
re.sub(a, '', 'From {{test}} and related to banana. From {{something}} else')

In [ ]:
# Remove Cognates
results = [[s, sentiment_data[s]['entry_ids']] for s in sentiment_data if re.search(r' ([Rr]elate|[Cc]ognate)[^.]*', s)]

# results = [
#     [s, sentiment_data[s]['entry_ids']]
#     for s in sentiment_data 
#     if 
#         re.search(r' [Rr]elate[^.]*', s) 
#         and not re.search(r' ([Cc]ognate|[Rr]elated)[^.]*', s)
# ]
results = sorted(results, key=lambda x: len(x[1]), reverse=True)
print(len(results), 'results')
for result in results:
    print(f'"{result[0]}"', result[1][:4])

In [ ]:
# evaluate this more (quoted words)
# [[s, sentiment_data[s]['entry_ids']] for s in sentiment_data if re.search(r'\"[^ ]+\"', s)]

# Manual Processing

In [2]:
wp = WikiProcessor(
    '/home/ubuntu/projects/wikiextractor/', 
#     channel='test', # Test makes self.test = true
    channel='dev', 
    store_intermediates=True,
)

In [ ]:
# After get_wikitext_part_array() fails # 160 s
# en_etys_dl = wp.cursor.d('SELECT * FROM entry_etymologies') # 22s
# en_conns_dl = wp.cursor.d('SELECT * FROM entry_connections') # 15s
# en_2_ety = {e['entry_id']:e['etymology_id'] for e in en_conns_dl} #0s
# etys_dl = wp.cursor.d('SELECT _id, word, language_name FROM etymologies') # 80s
# etys_dict = {e['_id']:e for e in etys_dl} # 1s
# wp.en_dict = {e['entry_id']:e['entry_number'] for e in en_conns_dl} # Needed later
# en_etys_dl_plus = [{**e, **etys_dict[en_2_ety[e['entry_id']]]} for e in en_etys_dl] # 3s

In [ ]:
# wikitext_part_array = wp.get_wikitext_part_array(en_etys_dl_plus) # 150s
# pickle.dump(wikitext_part_array, open('wikitext_part_array', 'wb+'))
# wikitext_part_array = pickle.load(open('wikitext_part_array', 'rb'))

In [ ]:
# all_connections, _ = wp.get_connections_from_wikitext_parts(wikitext_part_array) # 60s
# pickle.dump(all_connections, open('all_connections', 'wb+'))
all_connections = pickle.load(open('all_connections', 'rb'))

In [ ]:
wp.load_wl_2_id_values() # 90s
wp.load_language_dict() # 0s
node_connections = wp.get_nodes_from_connections(all_connections) # 60s

In [ ]:
excluded_tables = [ # all tables
    'ancestors',
    'common_words',
    # 'connection_sources',
    # 'connections',
    'database_errors',
    'entry_connections',
    'entry_definitions', 
    'entry_etymologies',
    'entry_pos',
    'entry_pronunciations',
    'etymologies',
    'kin',
    'languages',
    'merged_etymologies',
    'permanent_errors',
    'progeny',
    'random_etymologies',
    'transliterations',
    'wiktionary_page_dne'
]
refresh_tables(wp.cursor, exclude=excluded_tables)
wp.conn.commit()

In [ ]:
# Rerun from all_connections failure (3m)
data = wp.get_mysql_data_from_nodes(node_connections) # 9s
roots, descs, table_sources, entry_numbers, places = data; del data
wp.insert_unmatched_words_into_mysql()
wp.insert_connections_into_mysql(roots, descs, table_sources, entry_numbers, places)

In [ ]:
notify('done!')

In [ ]:
wp.conn.commit()

# Check bad connections

In [ ]:
wp = WikiProcessor(
    '/home/ubuntu/projects/wikiextractor/', 
    channel='dev', 
    store_intermediates=True,
)

In [ ]:
c = dev_conn # DEVELOPMENT
# root = 1003193; desc = 249159 # Proto-Italic magnos
root = 1708738; desc = 192563

# root=get_id(c,'animals','English'); desc=get_id(c,'breed','English')
query = f"""
    SELECT cs.root, cs.descendant, cs.table_source, e1.word, e1.language_name, e2.word, e2.language_name 
    FROM connection_sources cs 
    INNER JOIN etymologies e1 ON cs.root=e1._id 
    INNER JOIN etymologies e2 ON cs.descendant=e2._id 
    WHERE cs.root={root} """ + f""" AND cs.descendant={desc} """
data = c.cursor().e(query)
for d in data:
    print(f'({d[0]}, {d[1]}, {d[2]}) {d[3]}, {d[4]} => {d[5]}, {d[6]}')

In [ ]:
dev_cursor.e('ALTER TABLE connection_sources ADD COLUMN place tinyint NOT NULL AFTER entry_number;')

In [ ]:
dev_cursor.d('SELECT * FROM connection_sources LIMIT 1')

In [ ]:
enid = 2453428
g = get_entry_info(c, enid)
wikitext = next(iter(g['words'].values()))['entries'][enid]['wikitext']; wikitext #g
wp.get_connections_from_single_wikitext(wikitext)
# get_wikitext_parts_dict(**{'wikitext': wikitext, 'language_name': 'start_language', 'word': 'start_word', 'entry_id': 0})
# get_entry_connections(wikitext_parts)

In [ ]:
# Original source: http://devserver.etymologyexplorer.com:8888/notebooks/projects/wordkin-support/support/jupyter/Connection%20Manager.ipynb
# (34485, 6082, 322812)  potare, Latin => con-, Latin
# (1970733, 21180, ?) animals, English => breed, English
# (1983787, 297736, 660717) ksōn(w)ja, Albanian => ξένϝος, Ancient Greek
# (1983787, 297736, 1107985) ksōn(w)ja, Albanian => ξένϝος, Ancient Greek
# (1984171, 37444, 685629) kórčo, Mozarabic => cortex, Latin
# (5068, 3955, 174835) off, English => head, English
# (49619, 8, 14159) similar, English => -ary, English
# (201662, 15642, 807578) h₁epi, Proto-Indo-European => bonus, Latin
# (1991407, 241721, 1212552) manner, Latin => -o, Latin
# (37444, 5896, 819575) cortex, Latin => ex, Latin
# (187948, 118841, 37366) wet-, Proto-Indo-European => per, Proto-Indo-European
# (264389, 79947, 969676) yows-, Proto-Indo-European => ius, Latin
# (1987084, 75309, 893719) inscriptions, Proto-Norse => éǵh₂, Proto-Indo-European
# (1974448, 54135, 80636) me, Old Portuguese => ego, Latin
# (153206, 83775, 581774) só, Proto-Indo-European => is, Latin
# (1591662, 200433, 1221552) gręsti, Proto-Slavic => h₁ey-, Proto-Indo-European
# (1990026, 2242, 1134033) purificus, Latin => facio, Latin